In [15]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
import pandas as pd
import rdchiral
import itertools
import pickle
import sys
from tqdm import tqdm 
from rdchiral.main import rdchiralRun, rdchiralRunText, rdchiralReaction, rdchiralReactants
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
import matplotlib.pyplot as plt


sys.path.append('../../retrosynthesis/')
from common import *

### input and output

In [16]:
#input
yeast870_path = '../../../Data/model/yeast-GEM.yml'
yeast8U_path = '../../../Data/model/Yeast-MetaTwin.yml'
metabolites_info_to_GEM_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter.csv'

In [17]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)
yeast8U = cobra.io.load_yaml_model(yeast8U_path)

In [18]:
metabolites_info_to_GEM = pd.read_csv(metabolites_info_to_GEM_path)
metabolites_info_to_GEM

,new_met_smiles,sim_smile,ID,compartment
0,CC(N)C(O)=NC(CC(=O)O)C(=O)O,CC(N)C(=O)NC(CC(=O)O)C(=O)O,s_4120,c
1,CCCCCCCCCCCCCCCCCCCC=CC(=O)SCCNC(=O)CCNC(=O)C(...,CCCCCCCCCCCCCCCCCCCC=CC(=O)SCCNC(=O)CCNC(=O)C(...,s_2813,erm
2,C=C(OC1C=C(C(=O)O)C=CC1O)C(=O)[O-],C=C(OC1C=C(C(=O)O)C=CC1O)C(=O)O,s_0515,c
3,NC(=O)c1ncn(C2OC(COP(=O)(O)O)C(O)C2O)c1N=CN[C@...,NC(=O)c1ncn(C2OC(COP(=O)([O-])[O-])C(O)C2O)c1N...,s_0077,c
4,Nc1nc2c(c(=O)[nH]1)N=C(CNc1ccc(C(=O)NC(CCC(=O)...,Nc1nc2c(c(=O)[nH]1)N=C(CNc1ccc(C(=O)NC(CCC(=O)...,s_0625,c
...,...,...,...,...
4063,CCCCCCCCCCCCCCCCCC(=O)O[C@H](CO)COP(=O)([O-])O...,,sn_20,c
4064,CC(C)CC(N)C(=O)O,CC(C)CC(N)C(=O)O,s_1021,c
4065,CCCCCCC=CCCCCCCCC(=O)OCC(O)COP(=O)(O)OCC(N)C(=...,CCCCCCC=CCCCCCCCC(=O)OCC(O)COP(=O)(O)OCC(N)C(=O)O,s_3481,ce
4066,Nc1ccn(C2OC(COP(=O)([O-])OP(=O)(O)OP(=O)(O)O)C...,Nc1ccn(C2OC(COP(=O)(O)OP(=O)(O)OP(=O)(O)O)C(O)...,s_0539,c


# native

In [19]:
# L-glutamate --> L-glutamic 5-semialdehyde
# L-glutamate:s_0991
# L-glutamic 5-semialdehyde:s_0997
# P07275  YHR037W
# H2O + L-glutamate 5-semialdehyde + NAD+ = 2 H+ + L-glutamate + NADH
# EC:1.2.1.88 (UniProtKB | ENZYME| Rhea)

for i in yeast8.reactions:
    if 's_0991' in i.reaction and 's_0997' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_0991' in i.reaction and 's_0997' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})

r_0819 s_0180 + s_1266 --> s_0991 + s_0997
{'s_0180 [2-oxoglutarate]': -1, 's_0991 [L-glutamate]': 1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1266 [ornithine]': -1}
r_0819 s_0180 + s_1266 --> s_0991 + s_0997
{'s_0180 [2-oxoglutarate]': -1, 's_0991 [L-glutamate]': 1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1266 [ornithine]': -1}
rxn40939 s_0794 + s_0991 + s_1212 --> s_0803 + s_0997 + s_1207
{'s_0794 [H+]': -1, 's_0803 [H2O]': 1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1207 [NADP(+)]': 1, 's_1212 [NADPH]': -1}
rxn69409 s_0794 + s_0991 + s_1203 --> s_0803 + s_0997 + s_1198
{'s_0794 [H+]': -1, 's_0803 [H2O]': 1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1198 [NAD]': 1, 's_1203 [NADH]': -1}
rxn7558 s_0145 + s_0991 --> s_0997 + s_4173
{'s_0145 [2-acetamido-5-oxopentanoate]': -1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_4173 [N-acetyl-L-glutamate]': 1}


In [20]:
[x.id for x in yeast8U_del.reactions.get_by_id('rxn40939').genes]
# YHR037W

['YMR110C',
 'YPL098C',
 'YMR169C',
 'YOR323C',
 'YMR170C',
 'YER073W',
 'YHR037W',
 'YPL061W',
 'YBR006W',
 'YDR158W',
 'YOR374W',
 'YJR009C',
 'YGR192C',
 'YJL052W']

In [21]:
# kynurenate + L-glutamic --> L-kynurenine + 2-oxoglutarate
# s_2763(kynurenic acid/kynurenate ) +  s_0803 +  s_0991(L-glutamate) => s_1020(L-kynurenine) +  s_0180(2-oxoglutarate) 
# kynurenate:s_2763
# L-glutamic:s_0991
# s_0803
# L-kynurenine:s_1020
# 2-oxoglutarate:s_0180
# P38840_1  s_2763 +  s_0803 +  s_0991 => s_1020 +  s_0180
# 2-oxoglutarate + L-kynurenine = H2O + kynurenate + L-glutamate
# EC:2.6.1.7 (UniProtKB | ENZYME
# | Rhea)
# YHR137W

for i in yeast8.reactions:
    if 's_2763' in i.reaction and 's_1020' in i.reaction and 's_0180' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_2763' in i.reaction and 's_1020' in i.reaction and 's_0180' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})

rxn17810 s_0803 + s_0991 + s_2763 --> s_0180 + s_1020
{'s_0180 [2-oxoglutarate]': 1, 's_0803 [H2O]': -1, 's_0991 [L-glutamate]': -1, 's_1020 [L-kynurenine]': 1, 's_2763 [kynurenic acid]': -1}


In [22]:
[x.id for x in yeast8U_del.reactions.get_by_id('rxn17810').genes]
# YHR137W

['YDR111C',
 'YHR137W',
 'YER152C',
 'YLR027C',
 'YGR019W',
 'YGL202W',
 'YLR089C',
 'YNR058W',
 'YFL030W',
 'YOR184W',
 'YMR085W',
 'YER137C',
 'YHR208W',
 'YKL106W',
 'YDR204W',
 'YLR333C',
 'YIL087C',
 'YKL104C',
 'YIL116W',
 'YHR162W',
 'YLR201C',
 'YJL060W',
 'YMR084W',
 'YOL140W',
 'YGR243W',
 'YLR438W',
 'YJR148W',
 'YNR033W']

In [23]:
# L-lysine --> Cadaverine  ### YKL184W P08432 
# L-lysine:s_1025
# Cadaverine:s_4299

for i in yeast8.reactions:
    if 's_1025' in i.reaction and 's_4299' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_1025' in i.reaction and 's_4299' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})      

rxn4645 s_0794 + s_1025 --> s_0456 + s_4299
{'s_0456 [carbon dioxide]': 1, 's_0794 [H+]': -1, 's_1025 [L-lysine]': -1, 's_4299 [cadaverine]': 1}


In [24]:
[x.id for x in yeast8U.reactions.get_by_id('rxn4645').genes] 
#YKL184W

['YDR047W',
 'YER010C',
 'YDR380W',
 'YLL028W',
 'YKL184W',
 'YDR539W',
 'YKL088W',
 'YNL169C',
 'YKR072C',
 'YLR134W',
 'YNR043W',
 'YMR250W',
 'YGR087C',
 'YKR097W',
 'YNL052W',
 'YLR044C',
 'YOL052C',
 'YDL080C',
 'YEL021W',
 'YOR128C']

In [25]:
# L-homocysteine + S-methyl-L-methionine = H+ + 2 L-methionine  #Q08985 YPL273W
# L-methionine:s_1029
# S-methyl-L-methionine:s_1424
# L-homocysteine:s_1012

for i in yeast8.reactions:
    if 's_1029' in i.reaction and 's_1424' in i.reaction and 's_1012' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()}) 
print('===========')
for i in yeast8U.reactions:
    if 's_1029' in i.reaction and 's_1424' in i.reaction and 's_1012' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})      

rxn9778 s_1012 + s_1424 --> s_0794 + 2 s_1029
{'s_0794 [H+]': 1, 's_1012 [L-homocysteine]': -1, 's_1029 [L-methionine]': 2, 's_1424 [S-methyl-L-methionine]': -1}


In [26]:
[x.id for x in yeast8U.reactions.get_by_id('rxn9778').genes] 
#YPL273W

['YHR070W',
 'YER175C',
 'YER060W-A',
 'YJL125C',
 'YNL092W',
 'YOR074C',
 'YCR024C-A',
 'YGR027C',
 'YDR410C',
 'YDR198C',
 'YBR213W',
 'YBR236C',
 'YER056C',
 'YHR119W',
 'YGL136C',
 'YER060W',
 'YHR109W',
 'YMR160W',
 'YHR034C',
 'YNL063W',
 'YOL124C',
 'YOL125W',
 'YBR061C',
 'YPR186C',
 'YKL162C',
 'YOL093W',
 'YPR187W',
 'YGR002C',
 'YIL064W',
 'YPL030W',
 'YML008C',
 'YKR069W',
 'YJR137C',
 'YLR333C',
 'YOL103W',
 'YBR119W',
 'YBR115C',
 'YHR209W',
 'YMR257C',
 'YDR435C',
 'YDR140W',
 'YKR029C',
 'YGL186C',
 'YDR120C',
 'YBR241C',
 'YGR157W',
 'YGR006W',
 'YLR137W',
 'YDR105C',
 'YJR073C',
 'YPL266W',
 'YLL062C',
 'YER091C',
 'YOL096C',
 'YLR285W',
 'YJL056C',
 'YJL168C',
 'YDR497C',
 'YKL088W',
 'YLR172C',
 'YPL273W',
 'YML110C',
 'YOL141W',
 'YBR034C',
 'YOL020W',
 'YBL024W',
 'YJR044C',
 'YGL050W',
 'YHL036W',
 'YDL212W',
 'YDL200C',
 'YPL092W',
 'YDR440W',
 'YEL017C-A',
 'YDL201W',
 'YPL216W']